# Baseline model

In [2]:
import pandas as pd
import numpy as np
import gdown 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.sparse import hstack, vstack 

## Загрузка данных

In [3]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)

In [6]:
# Загрузка датафрейма с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c",  
               output="ratings.csv",
               quiet=False)
rating_df = pd.read_csv("ratings.csv", index_col=0)

# Метрика качества

In [8]:
def rel(r, p):
    return list(pd.Series(p).isin(r))

def mapk(relevant, predicted, k: int = 10):
    N = len(predicted)
    ru = np.array(list(map(rel, relevant, predicted))) # маска релевантных треков каждого польз-ля (k, N) 
    p_at_k = np.array([np.mean(ru[:,:i], axis=1) for i in range(1, k+1)]).T # Prescision at i для каждого user i = 1...k (N, k)
    nu = np.array(list(map(len, relevant))) # кол-во релевантных товаров каждого пользователя (N, ) 
    min_nu_k = 1/np.where(nu > k, k, nu) # min(k, nu)^(-1) для каждого user (N, )
    AP_at_k = np.sum(min_nu_k.reshape(N,1) * p_at_k * ru[:, :k], axis=1) # AP_at_k для каждого user (N, )
    MAP_at_k = np.mean(AP_at_k)
    
    return MAP_at_k


## Baseline 

В качестве рекомендации модель возвращает k самых популярных товаров

In [58]:
class Baseline():

    def __init__(self, rating_df: pd.DataFrame):
        self.rating = (rating_df
            .groupby("product_id", as_index=False)["rating"]
            .count()
            .sort_values(by="rating", ascending=False))

    def predict(self, n_recomend: int, k: int = 10):
        """
            Вычисление рекомендаций для n пользователей
        """
        return [self.recommend(k=k)] * n_recomend

    def recommend(self, k: int):
        """
            Расчет рекомендаций
        """
        return np.array(self.rating["product_id"][:k])


In [ ]:
# Пользователи с их релевантными покупками
relevant = test_df.groupby(['user_id']).agg({'product_id': 'unique'})['product_id']

In [36]:
# Инициализация модели
model = Baseline(rating_df=rating_df)
# Рекомендации
predict = model.predict(n_recomend=len(relevant), k=10)

In [67]:
# Оценка качества
mapk(relevant=relevant.to_list(), predicted=predict, k=10)